In [6]:
import os
import cv2
import numpy as np
from pathlib import Path

# Define dataset paths
dataset_path = "preprocessed_dataset"
subfolders = ["above40", "AdditionalAbove40", "age17-40", "AdditionalAge17-40"]
output_base_path = "edge_detected_dataset"

# Create output directories for Sobel and Canny
for subfolder in subfolders:
    Path(f"{output_base_path}/sobel/{subfolder}").mkdir(parents=True, exist_ok=True)
    Path(f"{output_base_path}/canny/{subfolder}").mkdir(parents=True, exist_ok=True)

def sobel_edge_detection(image, ksize=1):
    """
    Applies Sobel edge detection (sum of absolute X and Y derivatives).

    Args:
        image: Input image (can be color or grayscale).
        ksize: Kernel size for the Sobel operator.

    Returns:
        Grayscale edge magnitude image (uint8), or None if processing fails.
    """
    if image is None:
        print("  Error: Input image to sobel_edge_detection is None.")
        return None

    gray = None
    # Convert to grayscale if necessary
    if len(image.shape) == 3 and image.shape[2] == 3:  # Standard BGR
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    elif len(image.shape) == 2:  # Already grayscale
        gray = image.copy()
    elif len(image.shape) == 3 and image.shape[2] == 4:  # BGRA (e.g., PNG with alpha)
        try:
            gray = cv2.cvtColor(image, cv2.COLOR_BGRA2GRAY)
            print(f"  Info: Converted 4-channel image to grayscale.")
        except cv2.error as e:
            print(f"  Error converting 4-channel image to grayscale: {e}. Skipping.")
            return None
    else:  # Fallback for unexpected shapes
        print(f"  Warning: Image has an unexpected shape {image.shape}. Attempting BGR to grayscale conversion.")
        try:
            gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        except cv2.error as e:
            print(f"  Error converting image with shape {image.shape} to grayscale: {e}. Skipping.")
            return None
    
    if gray is None:  # Safeguard
        print("  Error: Grayscale conversion failed.")
        return None

    # Apply Sobel operator in X direction
    sobel_x = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=ksize)
    # Apply Sobel operator in Y direction
    sobel_y = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=ksize)

    # Convert the 64-bit float Sobel results to absolute values and then to uint8
    abs_sobel_x = cv2.convertScaleAbs(sobel_x)
    abs_sobel_y = cv2.convertScaleAbs(sobel_y)

    # Combine the absolute Sobel X and Y images
    sobel_combined = cv2.add(abs_sobel_x, abs_sobel_y)
    
    return sobel_combined

def apply_canny_edge_detection(image):
    """Apply Canny edge detection."""
    if len(image.shape) == 3:
        image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    return cv2.Canny(image, 100, 200)

def process_image(image_path, sobel_output_path, canny_output_path):
    """Process a single OPG scan for edge detection."""
    img = cv2.imread(image_path)
    if img is None:
        print(f"Failed to load {image_path}")
        return

    # Apply Sobel edge detection
    sobel_img = sobel_edge_detection(img)
    if sobel_img is not None:
        cv2.imwrite(sobel_output_path, sobel_img)

    # Apply Canny edge detection
    canny_img = apply_canny_edge_detection(img)
    cv2.imwrite(canny_output_path, canny_img)

def process_dataset():
    """Process all images in the dataset for edge detection."""
    for subfolder in subfolders:
        input_dir = Path(dataset_path) / subfolder
        sobel_output_dir = Path(output_base_path) / "sobel" / subfolder
        canny_output_dir = Path(output_base_path) / "canny" / subfolder

        for img_path in input_dir.glob("*.jpg"):  # Adjust extension if needed
            sobel_output_path = sobel_output_dir / img_path.name
            canny_output_path = canny_output_dir / img_path.name
            process_image(str(img_path), str(sobel_output_path), str(canny_output_path))

if __name__ == "__main__":
    process_dataset()
    print("Edge detection complete. Check 'edge_detected_dataset/sobel' and 'edge_detected_dataset/canny' folders.")

Edge detection complete. Check 'edge_detected_dataset/sobel' and 'edge_detected_dataset/canny' folders.
